# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: MLOps**

### Tecnológico de Monterrey

## **Fase 1 - Avance de proyecto - Equipo 7**
### **Problema de asignación de créditos: South German Dataset.**

## **Construcción, ajuste y evaluación de Modelos de Machine Learning**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score,KFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
#from imblearn.over_sampling import ADASYN
from lightgbm import LGBMClassifier
#from xgboost import XGBClassifier
from collections import Counter
from scipy.stats import randint
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import warnings
import pickle

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import learning_curve

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import f1_score

In [4]:
import yaml

with open(r'../params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

In [5]:
#print(config['data_load']['dataToModel'])

./data/DataToModel.csv


In [6]:
df = pd.read_csv(config['data_load']['dataToModel'])
#df = pd.read_csv(r"./data/Final_Model.csv")
#df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [7]:
X=df.drop(['credit_risk'],axis=1)
y=df['credit_risk']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def model(X=X_train, y=y_train):
    
    models=[]
    models.append(('GradientBoostingClassifier',GradientBoostingClassifier()))
    #models.append(('XGBClassifier',XGBClassifier(eval_metric='mlogloss')))
    models.append(('DecisionTreeClassifier',DecisionTreeClassifier()))
    models.append(('RandomForestClassifier',RandomForestClassifier()))
    models.append(('AdaBoostClassifier',AdaBoostClassifier()))
    models.append(('LogisticRegression',LogisticRegression()))
    models.append(('BaggingClassifier',BaggingClassifier()))
    models.append(('LGBMClassifier',LGBMClassifier()))
    models.append(('SGDClassifier',SGDClassifier()))
    models.append(('SVC',SVC()))
    
    results =[]
    names   =[]
    scoring ='accuracy'
    
    for name, model in models:
        #kfold = KFold(n_splits=10, random_state=42)
        kfold = RepeatedStratifiedKFold(n_splits=10, random_state=42, n_repeats=3)
        cross = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
        results.append(cross)
        names.append(name)
        print (f'Model:{name},Mean: {cross.mean()},Std Dev: {cross.std()}')
        print('---'*25)

In [10]:
model(X_train,y_train)

Model:GradientBoostingClassifier,Mean: 0.7959649122807018,Std Dev: 0.08449713653060288
---------------------------------------------------------------------------
Model:DecisionTreeClassifier,Mean: 0.7385087719298246,Std Dev: 0.08126278103418356
---------------------------------------------------------------------------
Model:RandomForestClassifier,Mean: 0.8276315789473684,Std Dev: 0.05557451969338563
---------------------------------------------------------------------------
Model:AdaBoostClassifier,Mean: 0.773859649122807,Std Dev: 0.08035566522162715
---------------------------------------------------------------------------
Model:LogisticRegression,Mean: 0.8089473684210527,Std Dev: 0.07305053336387131
---------------------------------------------------------------------------
Model:BaggingClassifier,Mean: 0.7859649122807018,Std Dev: 0.08117708983339843
---------------------------------------------------------------------------
[LightGBM] [Info] Number of positive: 138, number of neg

In [11]:
param_distribs = {'n_estimators': randint(low=1, high=500),
                  'max_depth': randint(low=1, high=10),
                  'max_features':randint(low=1,high=10),
                 }

rnd_RF = RandomizedSearchCV(RandomForestClassifier(),param_distributions=param_distribs,n_iter=10,cv=5,scoring='accuracy', 
                            random_state=42)

rnd_RF.fit(X_train,y_train)
rnd_RF.best_params_

{'max_depth': 3, 'max_features': 7, 'n_estimators': 244}

In [12]:
RFClassifier = RandomForestClassifier(max_depth=8, max_features=5, n_estimators=103, random_state=42)
RFClassifier.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, max_features=5, n_estimators=103,
                       random_state=42)

In [13]:
param_grid = [{'n_estimators': [3, 10, 30], 'max_depth': [2, 4, 6, 8],'booster': ['gbtree','dart'],
              'learning_rate':[0.3,0.5,0.01,0.1]}]

In [14]:
LGBClassifier=LGBMClassifier(random_state=42)
LGBClassifier.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 154, number of negative: 45
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 169
[LightGBM] [Info] Number of data points in the train set: 199, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.773869 -> initscore=1.230290
[LightGBM] [Info] Start training from score 1.230290
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

LGBMClassifier(random_state=42)

In [15]:
GBClassifier=GradientBoostingClassifier(random_state=42)
GBClassifier.fit(X_train,y_train)

GradientBoostingClassifier(random_state=42)

In [16]:
prediction_RF = RFClassifier.predict(X_test)

prediction_LGBM = LGBClassifier.predict(X_test)

prediction_GB = GBClassifier.predict(X_test)

In [17]:
print('Accuracy RForest...{}'.format(accuracy_score(y_test,prediction_RF)))
print('---'*25)
print('Accuracy LightGBM...{}'.format(accuracy_score(y_test,prediction_LGBM)))
print('---'*25)
print('Accuracy Gradient...{}'.format(accuracy_score(y_test,prediction_GB)))
print('---'*25)


print('Precision RForest...{}'.format(precision_score(y_test,prediction_RF)))
print('---'*25)
print('Precision LightGBM...{}'.format(precision_score(y_test,prediction_LGBM)))
print('---'*25)
print('Precision Gradient...{}'.format(precision_score(y_test,prediction_GB)))
print('---'*25)


print('Recall RForest...{}'.format(recall_score(y_test,prediction_RF)))
print('---'*25)
print('Recall LightGBM...{}'.format(recall_score(y_test,prediction_LGBM)))
print('---'*25)
print('Recall Gradient...{}'.format(recall_score(y_test,prediction_GB)))
print('---'*25)


print('Confusion Matrix RForest... \n {}'.format(confusion_matrix(y_test,prediction_RF)))
print('---'*25)
print('Confusion Matrix LightGBM... \n {}'.format(confusion_matrix(y_test,prediction_LGBM)))
print('---'*25)
print('Confusion Matrix Gradient... \n {}'.format(confusion_matrix(y_test,prediction_GB)))
print('---'*25)

Accuracy RForest...0.82
---------------------------------------------------------------------------
Accuracy LightGBM...0.8
---------------------------------------------------------------------------
Accuracy Gradient...0.78
---------------------------------------------------------------------------
Precision RForest...0.8695652173913043
---------------------------------------------------------------------------
Precision LightGBM...0.8837209302325582
---------------------------------------------------------------------------
Precision Gradient...0.9
---------------------------------------------------------------------------
Recall RForest...0.9302325581395349
---------------------------------------------------------------------------
Recall LightGBM...0.8837209302325582
---------------------------------------------------------------------------
Recall Gradient...0.8372093023255814
---------------------------------------------------------------------------
Confusion Matrix RForest... 


In [18]:
file = 'Credit_Data_RF.pkl'

pickle.dump(RFClassifier,open(file,'wb'))